####INSTALLS

In [ ]:
!pip install bitsandbytes
!pip install --upgrade transformers # Install latest version of transformers
!pip install --upgrade accelerate
!pip install sentencepiece
!pip install tqdm

In [1]:
import pandas as pd
import torch
import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments,
)
from torch.utils.data import Dataset

In [ ]:
pip install --quiet peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 2.1 MB/s eta 0:00:00


In [2]:
from peft import PeftModel, LoraConfig, TaskType, get_peft_model

In [ ]:
pip install --quiet wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 12.7 MB/s eta 0:00:00


In [3]:
import wandb

In [4]:
!wandb login

wandb: Currently logged in as: abhaymittal (abhaymittal-devops). Use `wandb login --relogin` to force relogin


In [ ]:
!git config --global credential.helper store

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# !pip install transformers bitsandbytes accelerate

####LOADING LLAMA 2

In [4]:
def load_model(
    model_name: str = "meta-llama/Llama-2-7b-chat-hf", load_on_gpu: bool = True   #meta-llama/Meta-Llama-3-8B-Instruct
) -> tuple[AutoModelForCausalLM, AutoTokenizer]:
    if load_on_gpu:
        config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=False,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
        )

        model = AutoModelForCausalLM.from_pretrained(
            model_name, low_cpu_mem_usage=True, quantization_config=config
        )
    else:
        print("Loading without quantization")
        model = AutoModelForCausalLM.from_pretrained(model_name, low_cpu_mem_usage=True)

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.pad_token_id = tokenizer.eos_token_id

    return model, tokenizer

In [5]:
# Load the model & tokenizer
model, tokenizer = load_model()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
query = [
    {
        "role": "system",
        "content": "You answer multiple choice questions with the correct letter answer. Your answer should be in this format: 'Correct Answer: {Letter}. Explanation: {Explanation}'",
    },
    {
        "role": "user", "content": """"Continuous exposure to indoor fluorescent lights is beneficial to the health of hamsters with heart disease.One group of hamsters exposed to continuous exposure to fluorescent lights has an average lifespan that is 2.5% longer than another one of the same species but living in a black wall.
Which of the following questions was the initial motivation for conducting the above experiment?
A.Can hospital light therapy be proved to promote patient recovery?
B.Which one lives longer, the hamster living under the light or the hamster living in the dark?
C.What kind of illness does the hamster have?
D.Do some hamsters need a period of darkness?
        """
    },
]

input_ids = tokenizer.apply_chat_template(
    query, add_generation_prompt=True, return_tensors="pt"
).to(model.device)

terminators = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

response = outputs[0][input_ids.shape[-1] :]

print(tokenizer.decode(response, skip_special_tokens=True))

 Correct Answer: B. Which one lives longer, the hamster living under the light or the hamster living in the dark?

Explanation: The experiment was conducted to compare the lifespan of hamsters exposed to continuous exposure to fluorescent lights with those living in a dark environment. The initial motivation for the experiment was to determine whether exposure to indoor fluorescent lights has a positive effect on the health of hamsters with heart disease, rather than exploring other questions such as the effectiveness of hospital light therapy (A), the type of illness the hamsters have (C), or whether some hamsters need a period of darkness (D).


In [ ]:
def inference(question):
  query = [
    {
        "role": "system",
        "content": "You answer multiple choice questions with the correct letter answer. Your answer should be in this format: 'Correct Answer: {Letter}. Explanation: {Explanation}'",
    },
    {
        "role": "user", "content": question
    },
  ]

  input_ids = tokenizer.apply_chat_template(
    query, add_generation_prompt=True, return_tensors="pt"
  ).to(model.device)

  terminators = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]

  outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
  )

  response = outputs[0][input_ids.shape[-1] :]

  return tokenizer.decode(response, skip_special_tokens=True)

In [ ]:
def checkResponse(response, answer):
  response = response.strip()
  if(response.lower()[:15] == "correct answer:"):
    if(response[16] == answer):
      return 1
    else:
      return 0
  return -1

In [ ]:
from tqdm.auto import tqdm

tqdm.pandas(desc="progress")

test_data = pd.read_csv("test.csv")
test_data["generated_response"] = test_data.progress_apply(lambda row: inference(row["query"]), axis=1)
test_data["prediction"] = test_data.progress_apply(lambda row: checkResponse(row["generated_response"], row["response"][0]), axis=1)


In [ ]:
test_data.to_csv("results.csv", index=False, header=True)

####FINE-TUNING

In [6]:
def tokenize_queries(tokenizer: AutoTokenizer, query: str, max_length: int = 1024) -> torch.Tensor:
    query = [
        {"role": "system", "content": "You answer multiple choice questions with the correct letter answer. Your answer should be in this format: 'Correct Answer: {Letter}. Explanation: {Explanation}'",
},
        {"role": "user", "content": query},
    ]

    input_ids = tokenizer.apply_chat_template(
        query,
        padding='max_length',
        max_length=max_length,
        add_generation_prompt=True,
        return_tensors="pt"
    )

    return input_ids

def tokenize_responses(tokenizer: AutoTokenizer, query: str, response: str, max_length: int = 1024) -> torch.Tensor:
    response = [
        {"role": "system", "content": "You answer multiple choice questions with the correct letter answer. Your answer should be in this format: 'Correct Answer: {Letter}. Explanation: {Explanation}'",
},
        {"role": "user", "content": query},
        {"role": "assistant", "content": response},
    ]

    labels = tokenizer.apply_chat_template(
        response,
        padding='max_length',
        max_length=max_length,
        add_generation_prompt=True,
        return_tensors="pt"
    )

    return labels

In [7]:
class RecipeDataset(Dataset):
    def __init__(self, data: pd.DataFrame):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {
            "input_ids": self.data.iloc[idx]["input_ids"][0],
            "labels": self.data.iloc[idx]["labels"][0],
        }

In [8]:
# Load the model & tokenizer
# model, tokenizer = load_model()

# Add Lora Matrices
lora_config = LoraConfig(
    r=2,
    lora_alpha=8,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.2,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

peft_model = get_peft_model(model, lora_config)

In [9]:
# Print the number of trainable parameters
print(f"Number of trainable parameters: {sum(p.numel() for p in peft_model.parameters() if p.requires_grad)}")

Number of trainable parameters: 1048576


In [17]:
# Load in the data
train_data = pd.read_csv("train.csv")
eval_data = pd.read_csv("eval.csv")

# Take the first 100 samples for training
train_data = train_data.head(100)

# Take the first 10 samples for evaluation
eval_data = eval_data.head(10)

In [18]:
# Tokenize the data
train_data["input_ids"] = train_data.apply(lambda row: tokenize_queries(tokenizer, row["query"]), axis=1)
train_data["labels"] = train_data.apply(lambda row: tokenize_responses(tokenizer, row["query"], row["response"]), axis=1)

eval_data["input_ids"] = eval_data.apply(lambda row: tokenize_queries(tokenizer, row["query"]), axis=1)
eval_data["labels"] = eval_data.apply(lambda row: tokenize_responses(tokenizer, row["query"], row["response"]), axis=1)

# Drop unecessary columns
train_data = train_data.drop(columns="query").drop(columns="response")
eval_data = eval_data.drop(columns="query").drop(columns="response")

In [12]:
wandb.init(
    project="dsl-final-project-1"
)

wandb: Currently logged in as: abhaymittal (abhaymittal-devops). Use `wandb login --relogin` to force relogin


In [14]:
# Setup trainer
training_args = TrainingArguments(
    output_dir="./checkpoints",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    learning_rate=1e-3,
    num_train_epochs=5,
    logging_steps=1,
    save_steps=10,
    log_level="info",
    report_to="wandb"
)

trainer = Trainer(model=peft_model,
    args=training_args,
    train_dataset=RecipeDataset(train_data),
    eval_dataset=RecipeDataset(eval_data)
)

In [ ]:
# Now train
trainer.train()

In [29]:
torch.cuda.empty_cache()

####LORA MODEL

In [ ]:
def load_lora_model(
    adapter_name: str,
    model_name: str = "meta-llama/Meta-Llama-3-8B-Instruct",
) -> tuple[AutoModelForCausalLM, AutoTokenizer]:
    config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=False,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name, low_cpu_mem_usage=True, quantization_config=config
    )

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    model = PeftModel.from_pretrained(model, adapter_name)

    return model, tokenizer

In [ ]:
# Load the model & tokenizer
model, tokenizer = load_lora_model("checkpoints/checkpoint-360")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-8B-Instruct/snapshots/e5e23bbe8e749ef0efcf16cad411a7d23bd23298/config.json
Model config LlamaConfig {
  "_name_or_path": "meta-llama/Meta-Llama-3-8B-Instruct",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 500000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.40.1",
  "use_cache": true,
  "vocab_size": 128256
}

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requireme

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing LlamaForCausalLM.

All the weights of LlamaForCausalLM were initialized from the model checkpoint at meta-llama/Meta-Llama-3-8B-Instruct.
If your task is similar to the task the model of the checkpoint was trained on, you can already use LlamaForCausalLM for predictions without further training.
loading configuration file generation_config.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-8B-Instruct/snapshots/e5e23bbe8e749ef0efcf16cad411a7d23bd23298/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": [
    128001,
    128009
  ],
  "max_length": 4096,
  "temperature": 0.6,
  "top_p": 0.9
}

loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-8B-Instruct/snapshots/e5e23bbe8e749ef0efcf16cad411a7d23bd23298/tokenizer.json
loading file added_tokens.json from cache at None
loading fi

In [ ]:
query = [
    {
        "role": "system",
        "content": "Greet me in a few words.",
    },
    {
        "role": "user", "content": "Hello!"
    },
]

input_ids = tokenizer.apply_chat_template(
    query, add_generation_prompt=True, return_tensors="pt"
).to(model.device)

terminators = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

response = outputs[0][input_ids.shape[-1] :]

print(tokenizer.decode(response, skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


assistantassistant
